In [ ]:
def generate_vbt_progress_graphs(df, athlete_id, pdf):
    """
    Generate line graphs for an athlete's historical VBT data to show progress in weight and speed.

    Args:
        df (pd.DataFrame): The dataset containing test data.
        athlete_id (int): Athlete ID.
        pdf (PdfPages): PDF object to save the plot.

    Returns:
        None
    """
    athlete_data = df[(df['ID'] == athlete_id) & (df['Workout Type'] == 'Strength')]
    if athlete_data.empty:
        print(f"No VBT data found for athlete ID {athlete_id}.")
        return

    test_types = ['Back Squat', 'Bench Press', 'Deadlift']
    sub_types = ['Starting Strength', 'Strength-Speed', 'Accelerative Strength', 'Absolute Strength']

    fig, axs = plt.subplots(len(test_types), len(sub_types), figsize=(16, 12), sharex='col')
    fig.subplots_adjust(hspace=0.4, wspace=0.3)

    athlete_name = f"{athlete_data['First Name'].iloc[0]} {athlete_data['Last Name'].iloc[0]}"

    # Iterate over Test Types and Sub-Types
    for i, test_type in enumerate(test_types):
        # Filter Test Numbers dynamically for this Test Type
        test_type_data = athlete_data[athlete_data['Test Type'] == test_type]
        unique_test_numbers = test_type_data['Test Number'].unique()
        unique_test_numbers.sort()  # Ensure sorted order

        if not test_type_data.empty:
            max_weight = test_type_data['Weight'].max()
            weight_limit = max(90, ceil(max_weight / 45) * 45)
        else:
            weight_limit = 90

        for j, sub_type in enumerate(sub_types):
            ax = axs[i, j]
            ax2 = ax.twinx()  # Create a twin Y-axis for speed

            # Filter data for the specific Test Type and Sub-Type
            sub_type_data = athlete_data[(athlete_data['Test Type'] == test_type) & 
                                         (athlete_data['Test Sub-Type'] == sub_type)]
            if sub_type_data.empty:
                ax.text(0.5, 0.5, "No Data", ha='center', va='center', fontsize=10, color='gray')
                ax.axis('off')
                continue

            # Sort data by Test Number for consistent plotting
            sub_type_data = sub_type_data.sort_values(by='Test Number')

            # Map Test Numbers to an index for uniformity within the Test Type
            test_number_mapping = {test_num: idx + 1 for idx, test_num in enumerate(unique_test_numbers)}
            sub_type_data['Mapped Test Number'] = sub_type_data['Test Number'].map(test_number_mapping)

            # Plot Weight and Speed over Mapped Test Numbers
            ax.plot(sub_type_data['Mapped Test Number'], sub_type_data['Weight'], marker='o', label='Weight (lbs)', color='blue', linewidth=2)
            ax2.plot(sub_type_data['Mapped Test Number'], sub_type_data['Speed'], marker='s', label='Speed (m/s)', color='orange', linewidth=2)

            # Annotate values for clarity
            for k in range(len(sub_type_data)):
                mapped_test_num = sub_type_data['Mapped Test Number'].iloc[k]
                weight = sub_type_data['Weight'].iloc[k]
                speed = sub_type_data['Speed'].iloc[k]
                phase = sub_type_data['Phase'].iloc[k] if 'Phase' in sub_type_data.columns else 'N/A'
                ax.text(mapped_test_num, weight, f"{weight:.1f}", ha='center', va='bottom', fontsize=8, color='blue')
                # Position Speed labels slightly lower
                ax2.text(mapped_test_num, speed - 0.05, f"{speed:.2f}", ha='center', va='top', fontsize=8, color='orange')
                # Label the phase on the x-axis
                ax.text(mapped_test_num, -0.1 * weight_limit, phase, ha='center', va='top', fontsize=8, rotation=45, color='black')

            # Set dynamic Y-axis limits for Weight
            ax.set_ylim(0, weight_limit)
            ax2.set_ylim(0, 1.3)

            # Add labels only for specific graphs
            if sub_type == 'Starting Strength' and i == 0:
                ax.set_ylabel("Weight (lbs)", fontsize=9, color='blue')
            if sub_type == 'Absolute Strength' and i == 0:
                ax2.set_ylabel("Speed (m/s)", fontsize=9, color='orange')

            # Aesthetics
            ax.set_title(f"{test_type} - {sub_type}", fontsize=10, fontweight='bold')
            ax.grid(alpha=0.3)

            # Customize ticks for readability
            ax.tick_params(axis='x', labelsize=8, length=0)  # Remove default tick labels
            ax.tick_params(axis='y', labelsize=8, colors='blue')
            ax2.tick_params(axis='y', labelsize=8, colors='orange')

            # Update x-tick labels to reflect mapped test numbers
            ax.set_xticks(range(1, len(unique_test_numbers) + 1))
            ax.set_xticklabels([str(test_num) for test_num in unique_test_numbers], fontsize=8)

    # Add overall title
    fig.suptitle(f"{athlete_name} - VBT Historical Progress", fontsize=16, fontweight='bold', y=0.98)

    # Save the plot to the PDF
    pdf.savefig(fig)
    plt.close(fig)
